In [74]:
import pandas as pd

In [75]:
# import data_NO2
data_NO2 = pd.read_csv('data_NO2.csv', delimiter=';')
data_NO2.head()

,datetime_utc,volume_demand,volume_production,spot_price
0,2015-12-31 23:00:00+00:00,4043.0,7818.0,16.39
1,2016-01-01 00:00:00+00:00,4010.0,7694.0,16.04
2,2016-01-01 01:00:00+00:00,3930.0,7437.0,15.74
3,2016-01-01 02:00:00+00:00,3839.0,7120.0,15.57
4,2016-01-01 03:00:00+00:00,3833.0,7041.0,15.47


In [76]:
#import kristiansand
kristiansand = pd.read_csv('kristiansand.csv')
kristiansand.head()

,time,temperature_2m (°C),apparent_temperature (°C)
0,2015-12-31T23:00,6.3,3.3
1,2016-01-01T00:00,6.1,3.4
2,2016-01-01T01:00,6.3,3.3
3,2016-01-01T02:00,6.5,3.0
4,2016-01-01T03:00,6.7,2.8


In [77]:
# import stavanger
stavanger = pd.read_csv('stavanger.csv')
stavanger.head()

,time,temperature_2m (°C),apparent_temperature (°C)
0,2015-12-31T23:00,7.3,1.4
1,2016-01-01T00:00,6.9,0.9
2,2016-01-01T01:00,7.0,0.9
3,2016-01-01T02:00,7.4,1.1
4,2016-01-01T03:00,8.0,1.8


In [78]:
# Convert datetime columns to datetime format and remove timezone from data_NO2
data_NO2['datetime_utc'] = pd.to_datetime(data_NO2['datetime_utc']).dt.tz_localize(None)
kristiansand['time'] = pd.to_datetime(kristiansand['time'])
stavanger['time'] = pd.to_datetime(stavanger['time'])

# Standardize column names for merging
data_NO2 = data_NO2.rename(columns={'datetime_utc': 'date'})
kristiansand = kristiansand.rename(columns={
    'time': 'date',
    'temperature_2m (°C)': 'krs_temp_2m',
    'apparent_temperature (°C)': 'krs_app_temp'
})
stavanger = stavanger.rename(columns={
    'time': 'date',
    'temperature_2m (°C)': 'stv_temp_2m',
    'apparent_temperature (°C)': 'stv_app_temp'
})

# Merge the DataFrames
merged_df = pd.merge(data_NO2, kristiansand, on='date', how='inner')
merged_df = pd.merge(merged_df, stavanger, on='date', how='inner')

merged_df.head()


,date,volume_demand,volume_production,spot_price,krs_temp_2m,krs_app_temp,stv_temp_2m,stv_app_temp
0,2015-12-31 23:00:00,4043.0,7818.0,16.39,6.3,3.3,7.3,1.4
1,2016-01-01 00:00:00,4010.0,7694.0,16.04,6.1,3.4,6.9,0.9
2,2016-01-01 01:00:00,3930.0,7437.0,15.74,6.3,3.3,7.0,0.9
3,2016-01-01 02:00:00,3839.0,7120.0,15.57,6.5,3.0,7.4,1.1
4,2016-01-01 03:00:00,3833.0,7041.0,15.47,6.7,2.8,8.0,1.8


In [79]:
# import gas_price
gas_price = pd.read_csv('gas_price.csv')
gas_price.head()

,Day,Natural Gas Price
0,09/13/2018,2.94
1,09/12/2018,2.98
2,09/11/2018,2.94
3,09/10/2018,2.90
4,09/7/2018,2.88


In [80]:
# Convert Day column in gas_price_df to datetime format
gas_price['Day'] = pd.to_datetime(gas_price['Day'])

# Convert date column in merged_df to date format (drop time) for merging
merged_df['date_only'] = pd.to_datetime(merged_df['date']).dt.date
gas_price['Day'] = gas_price['Day'].dt.date

# Merge the gas price data into merged_df based on the date
merged_df = pd.merge(merged_df, gas_price, left_on='date_only', right_on='Day', how='left')

# Drop unnecessary columns after merge
merged_df = merged_df.drop(columns=['date_only', 'Day'])

merged_df.head()

,date,volume_demand,volume_production,spot_price,krs_temp_2m,krs_app_temp,stv_temp_2m,stv_app_temp,Natural Gas Price
0,2015-12-31 23:00:00,4043.0,7818.0,16.39,6.3,3.3,7.3,1.4,2.28
1,2016-01-01 00:00:00,4010.0,7694.0,16.04,6.1,3.4,6.9,0.9,2.28
2,2016-01-01 01:00:00,3930.0,7437.0,15.74,6.3,3.3,7.0,0.9,2.28
3,2016-01-01 02:00:00,3839.0,7120.0,15.57,6.5,3.0,7.4,1.1,2.28
4,2016-01-01 03:00:00,3833.0,7041.0,15.47,6.7,2.8,8.0,1.8,2.28


In [81]:
# Drop the demand and production columns
merged_df = merged_df.drop(columns=['volume_demand', 'volume_production', 'krs_app_temp', 'stv_app_temp'])

# Rename Natural Gas Price column
merged_df = merged_df.rename(columns={'Natural Gas Price': 'gas_price'})

#Shift the gas_price column by 1
merged_df['gas_price'] = merged_df['gas_price'].shift(24)

# Set the 24 first gas_price value to the 25th value
merged_df['gas_price'].iloc[:24] = merged_df['gas_price'].iloc[24]

# Fill missing values in gas_price by carrying forward the last known value
merged_df['gas_price'] = merged_df['gas_price'].fillna(method='ffill')

merged_df.head()

/var/folders/hg/3hd1t1d17cxcqzk4bsnngqq40000gn/T/ipykernel_18038/1804404201.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  merged_df['gas_price'].iloc[:24] = merged_df['gas_price'].iloc[24]
/var/folders/hg/3hd1t1d17cxcqzk4bsnngqq40000g

,date,spot_price,krs_temp_2m,stv_temp_2m,gas_price
0,2015-12-31 23:00:00,16.39,6.3,7.3,2.28
1,2016-01-01 00:00:00,16.04,6.1,6.9,2.28
2,2016-01-01 01:00:00,15.74,6.3,7.0,2.28
3,2016-01-01 02:00:00,15.57,6.5,7.4,2.28
4,2016-01-01 03:00:00,15.47,6.7,8.0,2.28


In [82]:
# Save the updated DataFrame to a CSV file
output_path = 'data_NO2_boosted.csv'
merged_df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")


Data saved to data_NO2_boosted.csv


In [87]:
list = [1,2,3]
list[:0]

[]